In [2]:
import pandas as pd
from itertools import chain
import numpy as np

import holidays_co
from datetime import date, datetime

In [3]:
import pip
pip.main(["install", "openpyxl"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: openpyxl in c:\users\colac\anaconda3\envs\machinelearning\lib\site-packages (3.1.2)

Requirement already satisfied: et-xmlfile in c:\users\colac\anaconda3\envs\machinelearning\lib\site-packages (from openpyxl) (1.1.0)

0

In [4]:
meses = [
'ENERO',
'FEBRERO',
'MARZO',
'ABRIL',
 'MAYO',
 'JUNIO',
 'JULIO',
 'AGOSTO',
 'SEPTIEMBRE',
 'OCTUBRE',
 'NOVIEMBRE',
 'DICIEMBRE']
rangosHora = None
year = '2015'

Importación de datos de excel desde 2013 - 2019

In [5]:
def convertirAFormato24h(texto):
    if 'am' not in texto and 'pm' not in texto:
        return texto.replace(" ","")
    texto = texto.replace("-","")
    textoPartido = texto.split()
    if 'am'  in textoPartido:
        return textoPartido[0] + '-' + textoPartido[1]
    else:
        hDes = int(textoPartido[0])
        hHas = int(textoPartido[1])
        hHas = hHas + 12
        hDes = hDes + 12 if hHas > 14 else hDes
        
        return '{0}-{1} '.format(hDes, hHas)

In [6]:
def tratarHoja (hoja):   
    global rangosHora 
    hoja = hoja.iloc[1:, 1:len(hoja.columns)-1]
    hoja = hoja.T
    hoja.columns = hoja.iloc[0]
    colTotal = list(hoja.columns).index('TOTAL')
    hoja = hoja.iloc[1:,0:colTotal]
    rangosHora = [convertirAFormato24h(s) for s in hoja.columns[1:7]]
    hoja.columns = ['Dia'] + rangosHora
    return hoja

In [7]:
def resumirHoja(hoja,mes,year):
    global rangosHora
    hoja['Personas'] = [tuple(x) for x in (hoja[rangosHora]).values]
    hoja['Dia'] = hoja['Dia'].astype(int).astype(str)
    lens = list(map(len, hoja.Personas))
    res = pd.DataFrame({'Dia': np.repeat(hoja.Dia,lens),
                        'Mes':np.tile([mes],len(lens)*len(rangosHora)),
                        'Year':np.tile([year],len(lens)*len(rangosHora)),
                        'RangoHora': np.tile(rangosHora,len(lens)),
                        'Aforo': list(chain.from_iterable(hoja.Personas))})
    res[['Aforo']] = res[['Aforo']].apply(pd.to_numeric)
    res = res.assign(HoraDes=[f"{str(year).split('-')[0]}" for year in res['RangoHora']])
    res = res.assign(HoraHas=[f"{str(year).split('-')[1]}" for year in res['RangoHora']])
    res = res.assign(FechaDes=[pd.to_datetime(f"{str(year)}-{str(month)}-{str(day)} {str(hour)}:00", errors='coerce') for year, month, day, hour in zip(res['Year'], res['Mes'], res['Dia'], res['HoraDes'])])
    res = res.assign(FechaHas=[pd.to_datetime(f"{str(year)}-{str(month)}-{str(day)} {str(int(hour)-1)}:59", errors='coerce') for year, month, day, hour in zip(res['Year'], res['Mes'], res['Dia'], res['HoraHas'])])
    res = res.assign(Festivo=[holidays_co.is_holiday_date(fecha) if (not np.isnan(fecha.year)) and (fecha.year > 1970 or fecha.year < 99999) else False for fecha in res['FechaHas']])
    return res

In [8]:
def iterarAnio(year):    
    xlsx = pd.ExcelFile('./Aforo Biblioteca Proyecto Maestría/Ingreso a edificio '+year+'.xlsx', engine='openpyxl')
    hojas = [x.upper() for x in xlsx.sheet_names if x in meses]
    pdMesesResumidos = []
    for mes in hojas:
        hoja = pd.read_excel(xlsx, mes)
        hoja = tratarHoja(hoja)
        pdMesesResumidos.append(resumirHoja(hoja,meses.index(mes)+1, year))
    return pd.concat(pdMesesResumidos)

In [10]:
anios = list(range(2013,2019))
pdList = []
for anio in anios:
    pdList.append(iterarAnio(str(anio)))
resumenCompleto = pd.concat(pdList, ignore_index=True)
print(resumenCompleto)

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
resumenCompleto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9903 entries, 0 to 9902
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Dia        9903 non-null   object        
 1   Mes        9903 non-null   int32         
 2   Year       9903 non-null   object        
 3   RangoHora  9903 non-null   object        
 4   Aforo      9555 non-null   object        
 5   HoraDes    9903 non-null   object        
 6   HoraHas    9903 non-null   object        
 7   FechaDes   9897 non-null   datetime64[ns]
 8   FechaHas   9897 non-null   datetime64[ns]
 9   Festivo    9903 non-null   bool          
dtypes: bool(1), datetime64[ns](2), int32(1), object(6)
memory usage: 667.4+ KB


In [ ]:
resumenCompleto.describe().T

,count,mean,std,min,25%,50%,75%,max
Mes,9903.0,6.762193,3.252098,1.0,4.0,7.0,10.0,12.0


In [ ]:
resumenCompleto.Festivo.unique()

array([False,  True])

In [ ]:
holidays_co.is_holiday_date(resumenCompleto.FechaDes.dt.date[0])

False

In [ ]:
resumenCompleto.to_csv('salida.csv')